# Cuaderno 23: Modelo PESP para la calendarización de viajes

*Este cuaderno contiene material tomado del Trabajo de Integración Curricular de Liseth Recalde, en la carrera de Ingeniería Matemática en la Escuela Politécnica Nacional (Quito, 2022).*

En los problemas de calendarización de viajes se busca determinar horarios exactos de llegada y salida en las estaciones, para cada uno de los viajes de las diferentes líneas dentro de un sistema de transporte público, tomando en cuenta restricciones de duración para cada una de las actividades a realizar durante el recorrido. La calendarización puede ser *aperiódica*, si se establecen horarios para cada uno de los eventos de todos los viajes a lo largo del día, o *periódica*, si los horarios se establecen para un horizonte de tiempo menor (es decir, para un "período") y luego se repiten a lo largo del día.

Vamos a considerar un ejemplo de planificación periódica de horarios, bajo las siguientes condiciones:

* El horizonte de tiempo para la planificación es de $T=30$ minutos.
* El sistema de transporte público se representa mediante una Red de Transporte Público (*Public Transportation Network*, PTN), que es un grafo dirigido $(V,A)$, donde el conjunto de nodos $V$ representa a las estaciones del sistema, y el conjunto de arcos $A$ representa a las conexiones directas entre las diferentes estaciones.
* El sistema de transporte cuenta con líneas, cada una de las cuales se representa como un camino dirigido en la red de transporte, desde una estación de salida hasta una estación llegada. Asociada a cada línea está su frecuencia, es decir, la cantidad de viajes de esa línea que deben realizarse dentro del horizonte de tiempo. El conjunto de líneas se denota con $\mathscr{L}$. Para este ejemplo, asumiremos que la PTN contiene dos líneas, cada una con frecuencia igual a 1.
* Asumimos que la demanda de transporte ha sido previamente enrutada sobre la PTN y se expresa como un vector $\bar{w} \in \mathbb{Z}^A$ que indica el número de pasajeros que deben ser transportados sobre cada arco de la red dentro del horizonte de tiempo $T$.

Para formular el problema de calendarización de viajes, se define la Red Evento-Actividad (*Event-Activity Network*, EAN), $\mathscr{N}^0=(\mathscr{E}^0,\mathscr{A}^0)$ de la siguiente manera:

* El conjunto $\mathscr{E}^0$ de nodos de la red representa los __eventos__ del sistema de transporte: Para cada línea $l \in \mathscr{L}$ y para cada estación $v$ dentro del conjunto $V(l)$ de estaciones visitadas por $l$, se definen dos nodos $(v,\text{arr},l)$ y $(v,\text{dep},l)$, que representan los eventos de la llegada y la salida de $l$ a la estación $v$, respectivamente. Si la frecuencia de $l$ es mayor a 1, se define un par de nodos por cada viaje de $l$ programado dentro del horizonte de tiempo. Denotaremos por $ \mathscr{E}_{\text{arr}}= \left\{ \left ( v,\text{arr},l \right ): v\in V(l), l\in \mathscr{L} \right\}$ al conjunto de todos los eventos de llegada y por $\mathscr{E}_{\text{dep}}= \left\{ \left ( v,\text{dep},l \right ): v\in V(l), l \in \mathscr{L} \right\}$ al conjunto de todos los eventos de salida. Se tiene que $\mathscr{E}^0= \mathscr{E}_{\text{arr}}\cup \mathscr{E}_{\text{dep}}$.

* El conjunto $\mathscr{A}^0$ de los arcos de la red representa __actividades__ que los usuarios realizan durante su viaje en el sistema de transporte. Identificamos tres tipos distintos de actividades:
     * _Actividades de conducción_: Corresponden a los desplazamientos entre estaciones consecutivas dentro de una línea. Cada una de estas actividades se representa en la EAN por medio de un arco $( \left ( v_1,\text{dep},l \right ), \left ( v_2, \text{arr},l \right ))$ que conecta el nodo de salida de una línea $l \in \mathscr{L}$ desde una estación $v_1 \in V(l)$ con el nodo de llegada de $l$ a la siguiente estación $v_2 \in V(l)$ dentro de su recorrido. Denotaremos el conjunto de actividades (arcos) de conducción por $\mathscr{A}_{\text{drive}}= \left\{ \left ( \left ( v_1,\text{dep},l \right ), \left ( v_2, \text{arr},l \right ) \right ): \left ( v_1,v_2 \right ) \in A(l), l\in  \mathscr{L} \right\}$, donde $A(l)$ es el conjunto de arcos de la red de transporte que pertenecen a $l$.
     * _Actividades de espera_: Corresponden a las esperas dentro de un vehículo desde la llegada hasta la salida en cada una de las estaciones intermedias de un viaje. Cada una de estas actividades se representa por medio de un arco $( \left ( v,\text{arr},l\right ), \left ( v, \text{dep},l \right ))$ que conecta el nodo de llegada de una línea $l \in \mathscr{L}$ a una estación $v \in V(l)$ con el nodo de salida de $l$ desde la misma estación $v$. Denotaremos por $\mathscr{A}_{\text{wait}}= \left\{ \left ( \left ( v,\text{arr},l \right ), \left ( v, \text{dep},l \right ) \right ): v\in V(l), l\in\mathscr{L} \right\}$ al conjunto de actividades (arcos) de espera. 
     * _Actividades de transferencia_: Corresponden a los cambios de línea que los pasajeros pueden realizar durante sus viajes. Cada una de estas actividades se representa por medio de un arco $( \left ( v,\text{arr},l_1 \right ), \left ( v, \text{dep},l_2 \right ) )$ que conecta el nodo de llegada a una de una línea $l_1 \in \mathscr{L}$ a una estación $v \in V(l_1) \cap V(l_2)$ con el nodo de salida de otra línea $l_2 \in \mathscr{L}$ desde esa misma estación $v$.  Denotaremos por $\mathscr{A}_{\text{trans}}= \left\{ \left ( \left ( v,\text{arr},l_1 \right ), \left ( v, \text{dep},l_2 \right ) \right ): v \in V(l_1) \cap V(l_2), l_1,l_2 \in \mathscr{L}, l_1\neq l_2 \right\}$ al conjunto de actividades (arcos) de transferencia.
     * Finalmente, se define $\mathscr{A}^0 := \mathscr{A}_{\text{drive}}\cup\mathscr{A}_{\text{wait}}\cup\mathscr{A}_{\text{trans}}$.
     
En el problema de calendarización de viajes, se tienen definidos tres parámetros sobre la Red Evento-Actividad:

* Una cota inferior $L_a$ y una cota superior $U_a$ para la duración de cada actividad $a \in \mathscr{A}^0$; y,
* la cantidad $w_a$ de pasajeros que participan en cada actividad $a \in \mathscr{A}^0$ (también llamada *peso por pasajeros de la actividad $a$*). El vector $w \in \mathbb{Z}^{\mathscr{A}^0}$ puede obtenerse a partir del vector de demandas $\bar{w}$ de la PTN, distribuyendo los flujos de pasajeros sobre las distintas líneas.

El problema de calendarización periódica de viajes consiste en determinar, para cada evento $i \in \mathscr{E}^0$ un horario (es decir, un instante de tiempo) $\pi_i \in \{0, \ldots T-1\}$ de tal forma que la duración de cada actividad se encuentre entre de sus cotas inferior y superior. Los horarios se consideran expresados en módulo $T$, por lo que la condición anterior puede formularse como:
$$
L_a \leq \pi_j - \pi_i \mod T \leq U_a, \quad \forall a=(i,j) \in  \mathscr{A}^0.
$$
El objetivo es minimizar la suma de las duraciones de todas las actividades de la EAN, ponderada por sus pesos por pasajeros.

Para formular este problema como un programa lineal entero, definimos las siguientes variables de decisión:

- Variables enteras $\pi_i \in \{0, \ldots T-1\}, \, i \in \mathscr{E}^0$ que indican los horarios de los eventos; y, 
- Variables enteras $z_a, \, a\in \mathscr{A}^0$ que modelan la periodicidad del horario (es decir, las relaciones "módulo T").

Con esto, el modelo PESP (*Periodic Event Scheduling Problem*) para la calendarización de eventos, puede ser formulado de la siguiente manera:

\begin{align*}
  \min & \sum_{a=(i,j)\in \mathscr{A}^0}w_a \left( \pi_j - \pi_i + T z_a \right)\\
    \text{s.r.} \\
    &  \pi_j-\pi_i + T z_a \geq L_a, & \forall a=(i,j) \in \mathscr{A}^0,\\
     &\pi_j-\pi_i + T z_a \leq U_a, & \forall a=(i,j) \in \mathscr{A}^0,\\
     &\pi_i\in \left\{0, ..., T-1\right\}, & \forall i\in \mathscr{E}^0,\\
     &z_a\in \mathbb{Z}, & \forall a\in \mathscr{A}^0.
\end{align*}

La función objetivo mide el tiempo de duración de las actividades, ponderado por el número de pasajeros que participan en ellas. La primera familia de restricciones establece que la duración de cada actividad debe 
ser mayor o igual a su cota inferior, mientras que la segunda familia de restricciones establece que esta duración debe ser menor o igual a su cota superior.

Vamos a implementar este programa utilizando la interfaz Python de Gurobi.

In [ ]:
# D A T O S   D E L   E J E M P L O
import gurobipy as gp
from gurobipy import *
    
# Actividades, cotas inferiores, cotas superiores y pesos por pasajeros
AA, L, U, w = gp.multidict({
  ((1,"arr","l1"),(1,"dep","l1")):  (1, 3,45), #espera l1
  ((2,"arr","l1"),(2,"dep","l1")):  (2, 4,50), 
  ((3,"arr","l1"),(3,"dep","l1")):  (2, 4,60), 
  ((4,"arr","l1"),(4,"dep","l1")):  (1, 3,55), 
  ((5,"arr","l1"),(5,"dep","l1")):  (1, 3, 55), 
  ((6,"arr","l2"),(6,"dep","l2")):  (2, 4,30),  #espera l2
  ((7,"arr","l2"),(7,"dep","l2")):  (1, 3,50), 
  ((2,"arr","l2"),(2,"dep","l2")):  (2, 4,80), 
  ((3,"arr","l2"),(3,"dep","l2")):  (2, 4,40), 
  ((8,"arr","l2"),(8,"dep","l2")):  (2, 4, 45),     
  ((1,"dep","l1"),(2,"arr","l1")):  (5, 8,45), #conducción l1
  ((2,"dep","l1"),(3,"arr","l1")):  (6, 10,65), 
  ((3,"dep","l1"),(4,"arr","l1")):  (4, 6,85), 
  ((4,"dep","l1"),(5,"arr","l1")):  (7, 11,55), 
  ((6,"dep","l2"),(7,"arr","l2")):  (3, 5,30), #conducción l2
  ((7,"dep","l2"),(2,"arr","l2")):  (4, 6,50), 
  ((2,"dep","l2"),(3,"arr","l2")):  (6, 10,90), 
  ((3,"dep","l2"),(8,"arr","l2")):  (6, 9,45), 
  ((2,"arr","l1"),(2,"dep","l2")):  (3, 4,10), #transferencia 
  ((2,"arr","l2"),(2,"dep","l1")):  (3, 4,15),     
  ((3,"arr","l1"),(3,"dep","l2")):  (3, 4,5), 
  ((3,"arr","l2"),(3,"dep","l1")):  (3, 4,25),         
  })

# Horizonte de tiempo
T=30

# --- A PARTIR DE AQUÍ LOS DATOS SE CALCULAN EN FUNCIÓN DE LOS ANTERIORES --- 

# Eventos
EE = gp.tuplelist(set([e1 for (e1,e2) in AA] + [e2 for (e1,e2) in AA]))

# Lista de líneas
LL = gp.tuplelist(set([l for (i, s, l) in EE]))

# Diccionario de arcos de conducción por línea
Adrive = {l : [(e1, e2) for (e1, e2) in AA if e1[1]=="dep" and e2[1]=="arr" 
              and e1[2]==l and e2[2]==l] for l in LL}

# Diccionario de estaciones visitadas por línea
Vl = {}
for l in LL:
    # reconstruir arcos de conducción como pares entre estaciones
    Laux = tuplelist([(e1[0], e2[0]) for (e1, e2) in Adrive[l]])
    # seleccionar primera estación visitada por l
    i = [(i1, i2) for (i1, i2) in Laux if len(Laux.select('*', i1))==0][0][0] 
    ruta = [i]
    while len(Laux.select(i,'*'))>0:
        i = Laux.select(i,'*')[0][1]
        ruta.append(i)
    Vl[l] = ruta

    
# Arcos de la PTN
A = gp.tuplelist(set([(e1[0],e2[0]) for (e1,e2) in AA if e1[1]=="dep" and e2[1]=="arr" and e1[2]==e2[2]]))

# Nodos de la PTN
V = gp.tuplelist(set([v1 for (v1,v2) in A] + [v2 for (v1,v2) in A]))

# Verificación de datos procesados
print(LL)
print(Adrive)
print(Vl)
print(A)
print(V)

Podemos visualizar la red de transporte (PTN) empleando las bibliotecas `matplotlib` y `networkx`:

In [ ]:
# Visualización de la red PTN
import networkx as nx
import matplotlib.pyplot as plt
D = nx.DiGraph()
D.add_nodes_from(V)
node_labels= {i : str(i) for i in V}
D.add_edges_from(A)
plt.figure(figsize=(12,4))
pos = {1 : (1,6), 2 : (3.5,5), 3 : (6.5,5), 4 : (8.75,6), 5 : (8.75,4.5), 6:(1,5), 7 : (1,3), 8 : (8.75,3)}
nx.draw_networkx(D, pos, labels= node_labels, node_color='cyan', node_size=500)
#nx.draw_networkx_edge_labels(D, pos, edge_labels)
plt.show()
print('*** Se tienen definidas {} líneas: ***'.format(len(LL)))
for l in LL:
    print('{} : {}'.format(l, Vl[l]))

Adicionalmente, podemos visualizar la red evento-actividad (EAN):

In [ ]:
#Visualización de la red EAN
D = nx.DiGraph() #crea grafos dirigidos
D.add_nodes_from(EE) #Añadimos eventos
node_labels= {i : str(i[0]) for i in EE} #Etiquetas de nodos
# Separamos listas de nodos dep y arr para darles diferente forma.
nodos_dep= [v for v in D if v[1]=='dep']
nodos_arr= [v for v in D if v[1]=='arr']
# Lista de colores para usar en las líneas:
lista_color = ['lime', 'm','y', 'g',  'b', 'c', 'saddlebrown','indigo', 'orange', 'gray']
# Crear diccionario de colores para las lineas
color_lin = {LL[k] : lista_color[k % 10] for k in range(len(LL))} # a cada línea asigna un color de la lista de colores (hasta 10 líneas)
nodos_dep_color= [color_lin[v[2]] for v in nodos_dep] #v[2] indica la línea a la que pertenecen los nodos, color_lin[v[2]] indica el color de la línea según el diccionario color_lin
nodos_arr_color= [color_lin[v[2]] for v in nodos_arr]
D.add_edges_from(AA) #agregamos las actividades
edge_labels = {(i,j) :'w='+str(w[i,j]) + ', \n L=' + str(L[i,j])+', U=' + str(U[i,j]) for (i,j) in AA} 
plt.figure(figsize=(22,10)) #tamaño de la figura (20) ancho, (10) de alto.
#Posición de los eventos
pos={(1, 'arr', 'l1'): (0.5,6),   (1, 'dep', 'l1'): (2,6),   (2, 'arr', 'l1'): (3,5), 
     (2, 'dep', 'l1'): (4,5),     (3, 'arr', 'l1'): (6,5),   (3, 'dep', 'l1'): (7,5), 
     (4, 'arr', 'l1'): (8,6),     (4, 'dep', 'l1'): (9.5,6), (5, 'arr', 'l1'): (8,4.5), 
     (5, 'dep', 'l1'): (9.5,4.5), (6, 'arr', 'l2'): (0.5,3), (6, 'dep', 'l2'): (2,3), 
     (7, 'arr', 'l2'): (0.5,1),   (7, 'dep', 'l2'): (2,1),   (2, 'arr', 'l2'): (3,3), 
     (2, 'dep', 'l2'): (4,3),     (3, 'arr', 'l2'): (6,3),   (3, 'dep', 'l2'): (7,3), 
     (8, 'arr', 'l2'): (8.2,3),   (8, 'dep', 'l2'): (9.5,3)}

nx.draw_networkx_edge_labels(D, pos, edge_labels) # es para las etiquetas de los arcos. 
nx.draw_networkx_edges(D, pos, min_target_margin=30) #tamaño de las flechas de los arcos 
nx.draw_networkx_labels(D, pos, labels= node_labels)
nx.draw_networkx_nodes(D, pos, nodelist= nodos_dep, node_color=nodos_dep_color, node_shape='s', node_size=3000 ) #asigna forma a nodos dep
nx.draw_networkx_nodes(D, pos, nodelist= nodos_arr, node_color=nodos_arr_color, node_shape='o', node_size=3000 ) #asigna forma a nodos arr
plt.show()

La red anterior representa un sistema de transporte público que consta de:
- 2 líneas de transporte, identificadas con colores;
- estaciones identificadas con números del 1 al 8; 
- eventos de llegada de cada línea $l$ a cada estación $v \in V(l)$ (nodos círculos);
- eventos de salida de cada línea $l$ desde cada estación $v \in V(l)$ (nodos cuadrados);
- actividades de conducción, espera y transferencia, cada una marcada con su respectivo:
    - peso por pasajeros, $w$;
    - cota inferior de duración, $L$; y,
    - cota superior de duración, $U$.

Con estos datos, implementamos a continuación el modelo PESP:

In [ ]:
# I M P L E M E N T A C I Ó N   D E L   M O D E L O

# Crear el objeto modelo
m = Model('PESP')

# Crear las variables de asignación de horarios
pi = m.addVars(EE, name="pi", vtype=GRB.INTEGER, lb=0, ub=T-1)

# Crear las variables de periodicidad del horario
z = m.addVars(AA, name="z", vtype=GRB.INTEGER) 

# Crear la función objetivo
m.setObjective((quicksum(w[i,j]*(pi[j]-pi[i]+z[i,j]*T) for i,j in AA)), GRB.MINIMIZE)

# Añadir las restricciones
m.addConstrs((pi[j]-pi[i]+z[i,j]*T>=L[i,j] for i,j in AA), "Cota_inferior")
m.addConstrs((pi[j]-pi[i]+z[i,j]*T<=U[i,j] for i,j in AA), "Cota_superior")
# Esta restricción fija (arbitrariamente) el horario de un evento de referencia en 0. 
# el evento (6,"arr","l2") es tomado como referencia
m.addConstr((pi[(6,"arr","l2")]==0), "Comienzo 0") 

Resolvemos ahora el modelo:

In [ ]:
# O P T I M I Z A C I Ó N
m.optimize()

Mostramos los horarios de cada evento obtenidos en la solución:

In [ ]:
#R E S P U E S T A
for e in EE:
    print('{}= {}'.format(pi[e].varName, int(pi[e].x)))

Finalmente, graficamos la solución sobre la red EAN:

In [ ]:
D = nx.DiGraph() #crea grafos dirigidos
D.add_nodes_from(EE) #Añadimos eventos
node_labels= {i : '\u03C0 = ' + str(int(pi[i].x)) +'\n'+ str(i[0]) for i in EE} 
nodos_dep= [v for v in D if v[1]=='dep']
nodos_arr= [v for v in D if v[1]=='arr']
# lista de colores para usar en las líneas:
lista_color = ['lime', 'm','y', 'g',  'b', 'c', 'saddlebrown', 
               'indigo', 'orange', 'gray']
# crear diccionario de colores para las lineas
color_lin = {LL[k] : lista_color[k % 10] for k in range(len(LL))}
nodos_dep_color= [color_lin[v[2]] for v in nodos_dep]
nodos_arr_color= [color_lin[v[2]] for v in nodos_arr]
D.add_edges_from(AA) #agregamos las actividades
#agregamos las etiquetas a las actividades
edge_labels = {(i,j) : "z="+ str(int(z[(i,j)].x))+'\n'+'w='+str(w[i,j]) 
               + ', \n L=' + str(L[i,j])+', U=' + str(U[i,j]) for (i,j) in AA} 
plt.figure(figsize=(23,10)) #tamaño de la figura (20) ancho, (10) de alto.

#Posición de los eventos
pos={(1, 'arr', 'l1'): (0.5,6),   (1, 'dep', 'l1'): (2,6),   (2, 'arr', 'l1'): (3,5), 
     (2, 'dep', 'l1'): (4,5),     (3, 'arr', 'l1'): (6,5),   (3, 'dep', 'l1'): (7,5), 
     (4, 'arr', 'l1'): (8,6),     (4, 'dep', 'l1'): (9.5,6), (5, 'arr', 'l1'): (8,4.5), 
     (5, 'dep', 'l1'): (9.5,4.5), (6, 'arr', 'l2'): (0.5,3), (6, 'dep', 'l2'): (2,3), 
     (7, 'arr', 'l2'): (0.5,1),   (7, 'dep', 'l2'): (2,1),   (2, 'arr', 'l2'): (3,3), 
     (2, 'dep', 'l2'): (4,3),     (3, 'arr', 'l2'): (6,3),   (3, 'dep', 'l2'): (7,3), 
     (8, 'arr', 'l2'): (8.2,3),   (8, 'dep', 'l2'): (9.5,3)}

nx.draw_networkx_edges(D, pos, min_target_margin=30) #tamaño de las flechas de los arcos 
nx.draw_networkx_labels(D, pos, labels= node_labels)
nx.draw_networkx_nodes(D, pos, nodelist= nodos_dep, node_color=nodos_dep_color, node_shape='s', node_size=3000 )
nx.draw_networkx_nodes(D, pos, nodelist= nodos_arr, node_color=nodos_arr_color, node_shape='o', node_size=3000 )
nx.draw_networkx_edge_labels(D, pos, edge_labels) # es para las etiquetas de los arcos
plt.show()